# 4c. Improving the training loop

Now that we are able to compute the loss for our training data, we are able to train the model with the same couple of steps that we have encountered at the end of [**Notebook 2**](../2_Tensors/2b_Tensors_features_Solution.ipynb).

We will take this as a starting point to introduce the `torch.optim` package which provides us with the `Optimizer` API that greatly simplifies the training loop.

## Key concepts of this section

1. `Optimizer` API from the `torch.optim` package

In [ ]:
import random
import collections
import math
from typing import Tuple, List

import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
import seaborn as sns
sns.set(style="darkgrid")

from workshop import data
import helper

## Training loop so far

Going back to the last section of [**Notebook 4b**](4b_loss_functions_Solutions.ipynb), we finished by calculating the per batch loss.

In [ ]:
# Nothing new here, just repeating definitions for clarity ... don't do this at home!

class MnistDataSet:
    def __init__(self, train=True):
        subset = "training" if train else "test"
        self.x, self.y = torch.load(data.DATA_PATH / f"MNIST/processed/{subset}.pt")
        self.x = self.x.float()
    
    def __getitem__(self, key) -> Tuple[torch.Tensor, torch.Tensor]:
        return self.x[key], self.y[key]
    
    def __len__(self):
        return len(self.x)
    
class MnistDataLoader:
    def __init__(self, dataset, batch_size, shuffle, transform=None):
        self.dataset, self.batch_size, self.shuffle, self.transform = dataset, batch_size, shuffle, transform
    
    def __iter__(self):
        self.idx = list(range(len(self.dataset)))
        if self.shuffle:
            random.shuffle(self.idx)
        return self
    
    def __next__(self):
        if self.idx:
            batch, self.idx = self.idx[:self.batch_size], self.idx[self.batch_size:]
            x, y = self.dataset[batch]
            if self.transform:
                return self.transform(x, y)
            return x, y
        
        raise StopIteration()

class ImageNormalizer:
    def __init__(self, mean, std):
        self.mean, self.std = mean, std
    
    def __call__(self, x, y):
        return (x - self.mean).div_(self.std), y

class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.reshape(x.shape[0], -1)
        

def get_model():
    return torch.nn.Sequential(collections.OrderedDict([
        ("reshape", torch.nn.Flatten()),
        ("hidden", torch.nn.Linear(28*28,256)),
        ("sigmoid", torch.nn.Sigmoid()),
        ("output", torch.nn.Linear(256,10)),
      ]))

In [ ]:
train_dl = MnistDataLoader(MnistDataSet(train=True), 1024, True, ImageNormalizer(33.32, 78.57))
test_dl = MnistDataLoader(MnistDataSet(train=False), 1024, True, ImageNormalizer(33.32, 78.57))
ce = torch.nn.CrossEntropyLoss()

def accuracy(preds, target):
    return (preds.max(-1)[1] == target).float().mean()

# for x, y in train_dl:
#     preds = model(x)
#     print(ce(preds, y))
#     print(accuracy(preds, y))
        

In [**Notebook 2**](../2_Tensors/2b_Tensors_features_Solution.ipynb) we already saw such a loop when finding the parameters for a **linear fit** to some data by minimizing the **MSE**.

Let's us try to reproduce this algorithm with our current setup!

In [ ]:
def train_nn(model, n_epochs, lr):
    train_losses = np.array([])
    test_losses = np.array([])
    accuracies = np.array([])

    for epoch in range(n_epochs):
        for x, y in train_dl:
            train_loss = ce(model(x), y)
            train_loss.backward()
            train_losses = np.append(train_losses, train_loss.item())

            with torch.no_grad():
                for p in model.parameters():
                    p += - lr * p.grad
                    p.grad.data.zero_()

        test_loss, acc = evaluate_nn(model, test_dl)
        test_losses = np.append(test_losses, test_loss)
        accuracies = np.append(accuracies, acc)

        print(f"Epoch: {epoch} \t Training loss: {train_losses[-1]} \t Test loss: {test_losses[-1]} \t Test accurarcy: {accuracies[-1]}")
    return train_losses, test_losses, accuracies

def evaluate_nn(model, test_dl):
    preds = torch.tensor([])
    targets = torch.tensor([]).long()
    
    with torch.no_grad():
        for x, y in test_dl:
            targets = torch.cat([targets, y])
            preds = torch.cat([preds, model(x)])
        test_loss = ce(preds, targets)
    return test_loss.item(), accuracy(preds, targets).item()

def plot_metrics(train_losses, test_losses, accuracies):
    fig, (ax0, ax1) = plt.subplots(1, 2, sharex=True, figsize=(15,5))
    
    x = np.array(range(len(train_losses)))
    iterations_per_epoch = int(len(train_losses)/ len(test_losses))
    x_val = x[iterations_per_epoch - 1 :: iterations_per_epoch]
    ax0.plot(x, train_losses, label='train')
    ax0.plot(x_val, test_losses, label='test')
   
    ax0.legend()
    ax0.set_ylabel("Loss")
    ax0.set_xlabel("Iteration")
    
    ax1.set_ylabel("Accuracy")
    ax1.plot(x_val, accuracies)
    ax1.set_xlabel("Iteration")
    plt.tight_layout()

In [ ]:
model = get_model()
train_losses, test_losses, accuracies = train_nn(model, n_epochs=10, lr=0.01)
plot_metrics(train_losses, test_losses, accuracies)

## PyTorch Optimizers

The above is a very neat example. The actual training code is only the part from l.6 to l.15, while the rest is mainly for logging.

However, it looks like the part from l.12 to l.15 is very generic and it can certainly be refactored away:

In [ ]:
class Optimizer:
    def __init__(self, parameters, lr):
        self.parameters, self.lr = list(parameters), lr
    
    def step(self):
        with torch.no_grad():
            for p in self.parameters:
                p += - self.lr * p.grad
    
    def zero_grad(self):
        with torch.no_grad():
            for p in self.parameters:
                if p.grad is not None:
                    p.grad.data.zero_()
                
                
def train_nn(model, optim, n_epochs):
    train_losses = np.array([])
    test_losses = np.array([])
    accuracies = np.array([])

    for epoch in range(n_epochs):
        for x, y in train_dl:
            
            train_loss = ce(model(x), y)
            train_loss.backward()
            train_losses = np.append(train_losses, train_loss.item())

            optim.step()
            optim.zero_grad()

        test_loss, acc = evaluate_nn(model, test_dl)
        test_losses = np.append(test_losses, test_loss)
        accuracies = np.append(accuracies, acc)
        
        print(f"Epoch: {epoch} \t Training loss: {train_losses[-1]} \t Test loss: {test_losses[-1]} \t Test accurarcy: {accuracies[-1]}")
    return train_losses, test_losses, accuracies

In [ ]:
model = get_model()
train_losses, test_losses, accuracies = train_nn(model, Optimizer(model.parameters(), lr=0.01), n_epochs=10)
plot_metrics(train_losses, test_losses, accuracies)

torch.save(model, "../data/model.pt")

This basically introduces the core of what the [torch.optim.Optimizer](https://pytorch.org/docs/stable/_modules/torch/optim/optimizer.html#Optimizer) does. 

The `torch.optim` package, however, does not only offer an almost empty capsule for updating model parameters - it has all the deep learning batteries included: in our simple case, the weight update is performed with the **SGD** rule: 
$$\omega_t = \omega_{t-1} - \lambda * \nabla \omega$$

Much more advanced algorithms exist to perform the weight update, like **SGD with momentum**, **Adagrad**, **Adam**, ...

## Exercise 1:

Check the documentation for a couple of other optimizers and see if you can improve the model performance! 

In [ ]:
model = get_model()
train_losses, test_losses, accuracies = train_nn(model, torch.optim.AdamW(model.parameters(), lr=0.01), n_epochs=10)
plot_metrics(train_losses, test_losses, accuracies)

torch.save(model, "../data/model_optimized_with_adam.pt")

In [ ]:
class AdamOptimizer(Optimizer):
    def __init__(self, parameters, lr, beta1 = 0.9, beta2 = 0.999, epsilon=1e-8, wd=1e-2):
        self.parameters, self.lr, self.beta1, self.beta2, self.epsilon, self.wd = list(parameters), lr, beta1, beta2, epsilon, wd
        self.t, self.state = 0, [{"m": 0., "v": 0.} for _ in self.parameters]
    
    def step(self):
        self.t += 1
        with torch.no_grad():
            for s, p in zip(self.state, self.parameters):
                p.mul_(1-self.lr*self.wd)
                s["m"] = self.beta1 * s["m"] + (1-self.beta1)*p.grad
                s["v"] = self.beta2 * s["v"] + (1-self.beta2)*p.grad.pow(2)
                p += - self.lr * math.sqrt(1-self.beta2**self.t) / (1 - self.beta1**self.t) * s["m"] / (s["v"].sqrt() + self.epsilon)
                
model = get_model()
train_losses, test_losses, accuracies = train_nn(model, AdamOptimizer(model.parameters(), lr=0.01), n_epochs=10)
plot_metrics(train_losses, test_losses, accuracies)

torch.save(model, "../data/model_optimized_with_adam.pt")

In [ ]:
class LambOptimizer(Optimizer):
    def __init__(self, parameters, lr, beta1 = 0.9, beta2 = 0.999, epsilon=1e-8, wd=1e-2):
        self.parameters, self.lr, self.beta1, self.beta2, self.epsilon, self.wd = list(parameters), lr, beta1, beta2, epsilon, wd
        self.t, self.state = 0, [{"m": 0., "v": 0.} for _ in self.parameters]
    
    def step(self):
        self.t += 1
        with torch.no_grad():
            for s, p in zip(self.state, self.parameters):
                s["m"] = self.beta1 * s["m"] + (1-self.beta1)*p.grad
                s["v"] = self.beta2 * s["v"] + (1-self.beta2)*p.grad.pow(2)
                
                r = s["m"] / (s["v"].sqrt() + self.epsilon)* math.sqrt(1-self.beta2**self.t) / (1 - self.beta1**self.t)
                scale = r + self.wd*p
                p += - self.lr * scale * torch.norm(p).clamp(0., 10.) / torch.norm(scale)
                
model = get_model()
train_losses, test_losses, accuracies = train_nn(model, LambOptimizer(model.parameters(), lr=0.01), n_epochs=10)
plot_metrics(train_losses, test_losses, accuracies)

torch.save(model, "../data/model_optimized_with_lamb.pt")

## Exercise 2:

What other API is exposed by the `torch.optim` package?

In [ ]:
# TODO: Exercise 2

# Anwser is _LRScheduler

# Excercise 3:

Save only the model's parameters to *../data/model_params.pt*.

In [ ]:
torch.save(model.state_dict(), "../data/model_params.pt")

## Section summary

The `torch.optim` package provides useful APIs and state-of-the-art algorithms for performing weight updates and learning rate scheduling.